In [ ]:
from mdp import GridMDP
import time, random
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# 4x3 World
grid1 = [
    [0, 0, 0, +1],
    [0, None, 0, -1],
    [0, 0, 0, 0]
]
terminals1 = [(0,3), (1,3)]
mdp1 = GridMDP(grid1, terminals1, gamma=0.9)

# 5x5 World
grid2 = [
    [0, 0, 0, 0, +1],
    [0, None, None, 0, -1],
    [0, 0, 0, 0, 0],
    [0, None, 0, None, 0],
    [0, 0, 0, 0, 0]
]
terminals2 = [(0,4), (1,4)]
mdp2 = GridMDP(grid2, terminals2, gamma=0.9)


In [3]:
def value_iteration(mdp, gamma=0.9, theta=1e-3):
    U = {s: 0 for s in mdp.states}
    delta_history = []
    iteration = 0
    start_time = time.time()

    while True:
        iteration += 1
        delta = 0
        U_new = U.copy()
        for s in mdp.states:
            if s in mdp.terminals: 
                continue
            U_new[s] = max(
                sum(p * (mdp.R(s) + gamma * U[s1]) for (p, s1) in mdp.T(s,a))
                for a in mdp.actions(s)
            )
            delta = max(delta, abs(U_new[s] - U[s]))
        U = U_new
        delta_history.append(delta)
        if delta < theta: break

    policy = {s: None if s in mdp.terminals else
              max(mdp.actions(s), key=lambda a: sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)))
              for s in mdp.states}

    return U, policy, {"iterations": iteration, "delta_history": delta_history, "time": time.time()-start_time}


In [ ]:
def policy_evaluation(mdp, pi, gamma=0.9, theta=1e-3):
    U = {s: 0 for s in mdp.states}
    while True:
        delta = 0
        U_new = U.copy()
        for s in mdp.states:
            if s in mdp.terminals: continue
            a = pi[s]
            U_new[s] = sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a))
            delta = max(delta, abs(U_new[s]-U[s]))
        U = U_new
        if delta < theta: break
    return U

def policy_iteration(mdp, gamma=0.9, theta=1e-3):
    pi = {s: random.choice(mdp.actions(s)) if s not in mdp.terminals else None for s in mdp.states}
    iteration = 0
    start_time = time.time()
    while True:
        iteration += 1
        U = policy_evaluation(mdp, pi, gamma, theta)
        stable = True
        for s in mdp.states:
            if s in mdp.terminals: continue
            old = pi[s]
            pi[s] = max(mdp.actions(s), key=lambda a: sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)))
            if old != pi[s]: stable = False
        if stable: break
    return U, pi, {"iterations": iteration, "time": time.time()-start_time}


In [ ]:
def async_value_iteration(mdp, gamma=0.9, theta=1e-3):
    U = {s: 0 for s in mdp.states}
    iteration, delta_history = 0, []
    start_time = time.time()

    while True:
        iteration += 1
        delta = 0
        for s in mdp.states:  
            if s in mdp.terminals: continue
            u = U[s]
            U[s] = max(sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)) for a in mdp.actions(s))
            delta = max(delta, abs(u-U[s]))
        delta_history.append(delta)
        if delta < theta: break

    pi = {s: None if s in mdp.terminals else
          max(mdp.actions(s), key=lambda a: sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)))
          for s in mdp.states}
    return U, pi, {"iterations": iteration, "delta_history": delta_history, "time": time.time()-start_time}


In [ ]:
def prioritized_sweeping(mdp, gamma=0.9, theta=1e-3):
    U = {s: 0 for s in mdp.states}
    pq = []
    start_time = time.time()
    iteration = 0

    def residual(s):
        if s in mdp.terminals: return 0
        best = max(sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)) for a in mdp.actions(s))
        return abs(best - U[s])

    for s in mdp.states:
        heapq.heappush(pq, (-residual(s), s))

    while pq:
        iteration += 1
        _, s = heapq.heappop(pq)
        if residual(s) < theta: continue
        U[s] = max(sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)) for a in mdp.actions(s))
        heapq.heappush(pq, (-residual(s), s))

    pi = {s: None if s in mdp.terminals else
          max(mdp.actions(s), key=lambda a: sum(p*(mdp.R(s)+gamma*U[s1]) for (p,s1) in mdp.T(s,a)))
          for s in mdp.states}
    return U, pi, {"iterations": iteration, "time": time.time()-start_time}


In [ ]:
algos = {
    "VI": value_iteration,
    "PI": policy_iteration,
    "AsyncVI": async_value_iteration,
    "PS": prioritized_sweeping
}

results = []
for algo, func in algos.items():
    for mdp, name in [(mdp1,"4x3"), (mdp2,"5x5")]:
        U, pi, stats = func(mdp, gamma=0.9, theta=1e-3)
        results.append({
            "algo": algo,
            "map": name,
            "iterations": stats.get("iterations",None),
            "time": stats["time"]
        })

df = pd.DataFrame(results)
df
